# Fichier pour générer les soumissions et prédictions

### Se sert des fichiers .sav du model et de scaler

In [1]:
# librairies 
#----------------------------------------------
#----- Calculs 
import pandas as pd
import numpy as np
from numpy import mean, std
from math import sqrt, log
#save model 
import pickle 

### Récupération du test set d'évaluation Kaggle

In [21]:
# Lecture du fichier des données tests
#----------------------------------------------
PATH = '.' #modifier si besoin le chemin du fichier

# récupération du fichier contentant l'ensemble des données issues des requêtes
eval_set_kaggle_detailed = pd.read_csv(PATH + '//test_set_cleaned_addedFeatures.csv')
eval_set_kaggle_detailed.head()
eval_set_kaggle = pd.read_csv(PATH + '//test_set.csv')
idx = eval_set_kaggle ["index"]

In [22]:
# Assigner le bon type aux variables
#----------------------------------------------
#--- Convert to integer: 
int_list = ["date","avatar_id","hotel_id","stock","request_number","request_nb", "pib", "price_m2", "nb_tourists", "nb_hab_km2"] 
eval_set_kaggle_detailed[int_list] = eval_set_kaggle_detailed[int_list].astype(int) 
#--- Convert to categorical: 
eval_set_kaggle_detailed["city"] = pd.Categorical(eval_set_kaggle_detailed["city"],ordered=False)
eval_set_kaggle_detailed["language"] = pd.Categorical(eval_set_kaggle_detailed["language"],ordered=False)
eval_set_kaggle_detailed["mobile"] = pd.Categorical(eval_set_kaggle_detailed["mobile"],ordered=False)
eval_set_kaggle_detailed["parking"] = pd.Categorical(eval_set_kaggle_detailed["parking"],ordered=False)
eval_set_kaggle_detailed["pool"] = pd.Categorical(eval_set_kaggle_detailed["pool"],ordered=False)
eval_set_kaggle_detailed["children_policy"] = pd.Categorical(eval_set_kaggle_detailed["children_policy"],ordered=False)
eval_set_kaggle_detailed["group"] = pd.Categorical(eval_set_kaggle_detailed["group"],ordered=False)
eval_set_kaggle_detailed["brand"] = pd.Categorical(eval_set_kaggle_detailed["brand"],ordered=False)

### Tranformation de la variable quantitative 'stock' comme pour l'apprentissage

In [23]:
eval_set_kaggle_detailed["stock"]=eval_set_kaggle_detailed["stock"].map(lambda x: log(x+1))

In [24]:
# Variables explicatives
#-----------------------------------------------
# Liste des variables categorical: 

cat_list1 = ["city", "language", "brand","mobile","parking","pool","children_policy"] #avec brand
cat_list2 = ["city", "language", "group","mobile","parking","pool","children_policy"] #avec group
cat_list3 = ["city", "language", "brand", "group","mobile","parking","pool","children_policy"] #avec brand & group
cat_list4 = ["city", "brand", "group","mobile","parking","pool","children_policy"]

# Liste des variables quantitatives: 
quant_list1 = ["hotel_id", "stock","request_nb", "date", "pib", "price_m2", "nb_tourists", "nb_hab_km2"] #on a enlevé avatar_id et price (=variable à expliquer) #on a enlevé avatar_id et price (=variable à expliquer)

# Création de la dataframe train
eval_set_kaggle_detailed_pricingDum  = pd.get_dummies(eval_set_kaggle_detailed[cat_list4],drop_first = True)
eval_set_kaggle_detailed_pricingQuant = eval_set_kaggle_detailed[quant_list1]

dfC_eval_set_kaggle = pd.concat([eval_set_kaggle_detailed_pricingDum ,eval_set_kaggle_detailed_pricingQuant],axis=1)
dfC_eval_set_kaggle.columns

Index(['city_copenhagen', 'city_madrid', 'city_paris', 'city_rome',
       'city_sofia', 'city_valletta', 'city_vienna', 'city_vilnius',
       'language_belgian', 'language_bulgarian', 'language_croatian',
       'language_cypriot', 'language_czech', 'language_danish',
       'language_dutch', 'language_estonian', 'language_finnish',
       'language_french', 'language_german', 'language_greek',
       'language_hungarian', 'language_irish', 'language_italian',
       'language_latvian', 'language_lithuanian', 'language_luxembourgish',
       'language_maltese', 'language_polish', 'language_portuguese',
       'language_romanian', 'language_slovakian', 'language_slovene',
       'language_spanish', 'language_swedish', 'brand_Ardisson',
       'brand_Boss Western', 'brand_Chill Garden Inn', 'brand_Corlton',
       'brand_CourtYord', 'brand_Ibas', 'brand_Independant',
       'brand_J.Halliday Inn', 'brand_Marcure', 'brand_Morriot',
       'brand_Navatel', 'brand_Quadrupletree', 'brand_R

In [25]:
eval_set_kaggle_detailed

,hotel_id,stock,city,date,language,mobile,avatar_id,request_number,request_nb,group,brand,parking,pool,children_policy,pib,price_m2,nb_tourists,nb_hab_km2
0,161,3.850148,vilnius,21,romanian,0,1,1,1,Boss Western,J.Halliday Inn,1,0,0,68031,1469,2284,1432
1,187,3.496508,vilnius,21,romanian,0,1,1,1,Accar Hotels,Marcure,1,1,0,68031,1469,2284,1432
2,279,2.564949,vilnius,21,romanian,0,1,1,1,Independant,Independant,1,0,0,68031,1469,2284,1432
3,395,2.397895,vilnius,21,romanian,0,1,1,1,Accar Hotels,Ibas,0,0,0,68031,1469,2284,1432
4,488,3.761200,vilnius,21,romanian,0,1,1,1,Accar Hotels,Safitel,1,1,1,68031,1469,2284,1432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,987,0.693147,rome,5,irish,0,794,843,3,Accar Hotels,Ibas,1,0,0,1996934,3044,38419,2213
6640,26,0.693147,vienna,1,irish,1,794,844,4,Accar Hotels,Marcure,1,0,0,468046,6550,15091,4607
6641,263,0.000000,vienna,1,irish,1,794,844,4,Boss Western,Boss Western,1,0,0,468046,6550,15091,4607
6642,456,0.000000,vienna,1,irish,1,794,844,4,Yin Yang,Ardisson,1,0,0,468046,6550,15091,4607


### Scaler tranformation & chargement du modèle

Les noms sont à adapter : charger les modèles et standard scaler correspondant(date)

In [26]:
model_folder = '//models_sav/'
scaler = pickle.load(open(PATH + model_folder + '12_11_One_hot_encoding_Standard_Scaler_Data.sav', 'rb')) #chargement
Xr_eval = scaler.transform(dfC_eval_set_kaggle)
# transformation de Xr_train et Xr_test (numpy array) en dataframe
Xr_eval = pd.DataFrame(Xr_eval, index=dfC_eval_set_kaggle.index, columns=dfC_eval_set_kaggle.columns)

In [27]:
#filename = 'X_gboost_tuned_model.sav' # variable
filename = '12-11_One_hot_encoding_lXgboost.sav'
loaded_model = pickle.load(open(PATH + model_folder + filename, 'rb'))

### Prédiction et enregistrement csv

In [28]:
pred_eval = loaded_model.predict(Xr_eval)
predict = pd.DataFrame(list(zip(idx, pred_eval**2)), columns=['index', 'price']) #### Mettre la prediction au carré car tranfor sqrt dans la transformation des données
predict.to_csv('submission.csv', index=False)
print(predict)

      index       price
0         0  105.987916
1         1  137.437407
2         2   55.203206
3         3   56.174096
4         4  218.144359
...     ...         ...
6639   6639   77.625731
6640   6640  247.713333
6641   6641  158.688774
6642   6642  188.878316
6643   6643  148.304080

[6644 rows x 2 columns]


In [29]:
loaded_model

LGBMRegressor(colsample_bytree=0.7, learning_rate=0.5, max_depth=10,
              n_estimators=2000, n_jobs=2, nthread=4, subsample=0.9)